In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from ema_workbench import load_results
from ema_workbench import (Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator, 
                           ema_logging, Constant, Scenario, Policy, CategoricalParameter, IntegerParameter,
                           perform_experiments, SequentialEvaluator, Constraint)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)
from ema_workbench.util import ema_logging, utilities
import time
from problem_formulation import get_model_for_problem_formulation
#from signal import signal, SIGPIPE, SIG_DFL
#signal(SIGPIPE, SIG_DFL)

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [5]:
results = pd.read_csv("results/results_reevaluation.csv")

In [7]:
results.drop(['Unnamed: 0'], axis=1, inplace=True)

In [8]:
results

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model,All Costs,Expected Number of Deaths,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths
0,76,262.700608,10.0,0.640277,141.966071,1.5,0.206812,180.506684,1.0,0.385086,...,1,2,0,4,dikesnet,9.537893e+08,0.000000,0.000000e+00,5.397251e+07,0.000000
1,106,306.403307,1.0,0.525580,126.403024,10.0,0.132563,178.796506,1.0,0.513056,...,1,2,1,4,dikesnet,9.537893e+08,0.000000,0.000000e+00,5.397251e+07,0.000000
2,97,209.753324,1.5,0.408170,308.826473,10.0,0.314795,270.550638,1.0,0.197602,...,1,2,2,4,dikesnet,9.537893e+08,0.000000,0.000000e+00,5.397251e+07,0.000000
3,53,240.358920,10.0,0.939596,49.019791,1.5,0.434024,247.644907,1.0,0.964298,...,1,2,3,4,dikesnet,9.537893e+08,0.000000,0.000000e+00,5.397251e+07,0.000000
4,8,305.469762,10.0,0.312076,96.172885,1.0,0.751104,345.582450,1.5,0.304617,...,1,2,4,4,dikesnet,9.537893e+08,0.000000,0.000000e+00,5.397251e+07,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,121,177.066954,1.5,0.352684,95.321026,1.5,0.690870,341.099764,10.0,0.258666,...,2,3,995,33,dikesnet,4.912596e+08,0.000000,0.000000e+00,3.725520e+07,0.000000
7996,48,272.273384,1.0,0.375960,47.997302,1.0,0.054370,287.737349,10.0,0.171085,...,2,3,996,33,dikesnet,5.020538e+08,0.001039,0.000000e+00,3.725520e+07,0.000000
7997,70,221.377266,10.0,0.849412,264.339809,1.5,0.612174,180.926475,1.5,0.043990,...,2,3,997,33,dikesnet,4.912596e+08,0.000000,0.000000e+00,3.725520e+07,0.000000
7998,40,155.103737,1.5,0.062811,77.309677,10.0,0.470226,292.422968,1.0,0.254511,...,2,3,998,33,dikesnet,8.670009e+08,0.049281,3.589998e+08,3.725520e+07,0.047613


### Robustness metrics

In this phase we evaluate the robustness of the set of policies. Different robustness metrics capture different aspects and trade-offs of the policies. Moreover, preferences of policy makers might change over time, so it is essential to use various metrics. We have included the following metrics:


Firstly, we define the functions

In [9]:
def s_to_n(data, direction):
    mean = np.mean(data)
    std = np.std(data)
    
    if direction==ScalarOutcome.MAXIMIZE:
        return mean/std
    else:
        return mean*std

In [10]:
def calculate_regret(data, best):
    return np.abs(best-data)